## Import Libraries

In [24]:
import os
import pandas as pd
import torch
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score


# Load and Split Dataset

In [25]:
# Load the multilabel data
data = pd.read_csv('LandUse_Multilabeled.txt', sep='\t')
X = data.iloc[:, 0]  # Image names
y = data.iloc[:, 1:]  # Labels

# Split into train, validation, and test sets
# No stratification is applied because it gave a message that some classes only occur once
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Define Dataset Class

In [26]:
class UCMercedDataset(Dataset):
    def __init__(self, image_names, labels, image_dir, transform=None):
        self.image_names = image_names
        self.labels = labels
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        # Extract the subfolder name by removing the last two characters
        subfolder = self.image_names.iloc[idx][:-2]  # e.g., 'tenniscourt' from 'tenniscourt00'
        img_path = os.path.join(self.image_dir, subfolder, self.image_names.iloc[idx])
        image = datasets.folder.default_loader(img_path)  # Load image
        label = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

# Preprocess Images

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = UCMercedDataset(X_train, y_train, 'Images', transform)
val_dataset = UCMercedDataset(X_val, y_val, 'Images', transform)
test_dataset = UCMercedDataset(X_test, y_test, 'Images', transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Load Pretrained Models

In [ ]:
'''
from timm import create_model

# Load DINO v2 ResNet 50
resnet50 = create_model('dino_resnet50', pretrained=True, num_classes=y.shape[1])

# Load DINO v2 ViT-S/8
vit_s8 = create_model('dino_vits8', pretrained=True, num_classes=y.shape[1])
'''


# Define Training Loop

In [ ]:
'''
def train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
'''


# Train and Evaluate Models

In [ ]:
'''
# Train ResNet 50
print("Training ResNet 50...")
train_model(resnet50, train_loader, val_loader)

# Train ViT-S/8
print("Training ViT-S/8...")
train_model(vit_s8, train_loader, val_loader)
'''

# Evaluate Models

In [ ]:
'''

def evaluate_model(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = torch.sigmoid(model(images))
            preds = (outputs > 0.5).float()
            all_labels.append(labels.cpu())
            all_preds.append(preds.cpu())

    all_labels = torch.cat(all_labels)
    all_preds = torch.cat(all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

# Evaluate ResNet 50
print("Evaluating ResNet 50...")
evaluate_model(resnet50, test_loader)

# Evaluate ViT-S/8
print("Evaluating ViT-S/8...")
evaluate_model(vit_s8, test_loader)
'''